#  WEBSCRAPPING autoscout24

- Importe de librerías necesarias

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.by import By

from tqdm import tqdm

import re
import time

import pandas as pd

- Creamos la configuración del navegador

In [2]:
c_options = webdriver.ChromeOptions()
c_options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=c_options)

- Establecemos las configuraciones de la URL para buscar con los diferentes filtros

In [3]:
base_url = "https://www.autoscout24.es/lst/tesla/"
tesla_models = ["model-3", "model-s", "model-x", "model-y"]

fixed_params = (
    "atype=C&damaged_listing=exclude&desc=0&powertype=kw"
    "&search_id=a1ps9r8ye6&sort=standard&source=homepage_search-mask&ustate=N%2CU"
)

bcol_values = ['bc_azul', 'bc_rojo', 'bc_blanco', 'bc_negro', 'bc_gris']
cy_values = ["E", "F", "I"]

country_mapping = {
    "E": "España",
    "F": "Francia",
    "I": "Italia"
}

results = []

accept_cookies = False

patron_tesla = re.compile(r'\bTesla\b', re.IGNORECASE)
patron_modelo = re.compile(r'\bModel (3|S|X|Y)\b', re.IGNORECASE)


- Realizamos el Scrapeo de las diferentes páginas

In [4]:
try:
    for model in tqdm(tesla_models):
        for bcol in bcol_values:
            for cy in cy_values:
                query_params = f"cy={cy}"
                full_url = f"{base_url}{model}/{bcol}?{fixed_params}&{query_params}"
                driver.get(full_url)

                if not accept_cookies:
                    try:
                        wait = WebDriverWait(driver, 10)
                        cookie_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "_consent-accept_1lphq_114")))
                        cookie_button.click()
                        accept_cookies = True
                    except TimeoutException:
                        print("No se encontraron cookies para aceptar.")

                while True:
                    time.sleep(3)
                    car_cards = driver.find_elements(By.TAG_NAME, 'article')
                    if len(car_cards) > 0:
                        for car in car_cards:
                            try:
                                marca_texto = car.find_element(By.TAG_NAME, 'h2').text
                                modelo = patron_modelo.search(marca_texto)
                            except NoSuchElementException:
                                marca_texto = "No disponible"
                                modelo = None

                            try:
                                descripcion = car.find_element(By.CLASS_NAME, 'ListItem_subtitle__VEw08').text
                            except NoSuchElementException:
                                descripcion = "No disponible"

                            try:
                                precio = car.find_element(By.CSS_SELECTOR, '[data-testid="regular-price"]').text
                                precio_limpio = re.sub(r'[^\d.]', '', precio)
                                precio = float(precio_limpio.replace('.', ''))
                            except NoSuchElementException:
                                precio = "No disponible"

                            try:
                                kilometraje = car.find_element(By.CSS_SELECTOR, '[data-testid="VehicleDetails-mileage_road"]').text
                                kilometraje_limpio = re.sub(r'[^\d.]', '', kilometraje)
                                kilometraje = int(kilometraje_limpio.replace('.', ''))
                            except NoSuchElementException:
                                kilometraje = "No disponible"

                            try:
                                fecha = car.find_element(By.CSS_SELECTOR, '[data-testid="VehicleDetails-calendar"]').text
                            except NoSuchElementException:
                                fecha = "No disponible"

                            try:
                                velocidad = car.find_element(By.CSS_SELECTOR, '[data-testid="VehicleDetails-speedometer"]').text
                            except NoSuchElementException:
                                velocidad = "No disponible"

                            pais_nombre = country_mapping.get(cy, "Desconocido")
                            modelo_final = modelo.group(0) if modelo else "No disponible"

                            results.append({
                                "Modelo": modelo_final,
                                "Precio": precio,
                                "Color": bcol.split('_')[1].capitalize(),
                                "Pais": pais_nombre,
                                "Kilometraje": kilometraje,
                                "Velocidad": velocidad
                            })

                        try:
                            next_button = driver.find_elements(By.CLASS_NAME, "prev-next")[1]
                            if "pagination-item--disabled" in next_button.get_attribute("class"):
                                break
                            else:
                                next_button.click()
                        except NoSuchElementException:
                            break
                    else:
                        break


finally:
    driver.quit()


100%|██████████| 4/4 [12:59<00:00, 194.79s/it]


In [5]:
df_tesla = pd.DataFrame(results)
df_tesla.to_csv("tesla_autoscout24.csv", index=True, encoding="utf-8-sig")

df_tesla.tail(20)

,Modelo,Precio,Color,Pais,Kilometraje,Velocidad
1345,Model Y,36200.0,Negro,Italia,26000,88 kW (120 CV)
1346,Model Y,35900.0,Negro,Italia,45000,88 kW (120 CV)
1347,Model Y,20000.0,Negro,Italia,8000,88 kW (120 CV)
1348,Model Y,50900.0,Gris,España,5000,378 kW (514 CV)
1349,Model Y,35990.0,Gris,Francia,54000,379 kW (515 CV)
1350,Model Y,39999.0,Gris,Francia,29103,350 kW (476 CV)
1351,Model Y,41480.0,Gris,Francia,46600,258 kW (351 CV)
1352,Model Y,38000.0,Gris,Francia,70000,350 kW (476 CV)
1353,Model Y,59900.0,Gris,Francia,2500,353 kW (480 CV)
1354,Model Y,47990.0,Gris,Francia,36000,351 kW (477 CV)
